In [9]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/user/project/python/rmsKit")
import os
import torch
from lattice import KH
# # os.environ["CUDA_VISIBLE_DEVICES"] = ""
# # os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import rms_torch
import numpy as np
# u = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
u = np.load("/home/user/project/python/rmsKit/array/torch/KH/3site/smel/"+
        "Jx_1_Jy_1_Jz_1_hx_0_hz_0/e_100_lr_0.001/loss_7.985083/u/0.npy")
device = torch.device("cuda")
p = dict(
    Jx=1,
    Jy=1,
    Jz=1,
    hx=0,
    hz=0,
)
H = KH.system([2, 2], "3site", p)
# Create an instance of the CustomModel class
E, V = np.linalg.eigh(H)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
H_gpu = torch.tensor(H, dtype=torch.float64, device=device)
model = rms_torch.UnitaryRieman(H.shape[0], 8, device=device, u0=u).to(device)
loss_smel = rms_torch.SystemMinimumEnergyLoss(H, device=device).to(device)
loss_sel = rms_torch.SystemEnergyLoss(H, device=device).to(device)
compiled_model = torch.compile(model, dynamic = False, fullgraph=True)


## Before optimization

In [23]:
loss_smel.initializer(torch.eye(4096, device=device, dtype = torch.float64))
sel = loss_sel(torch.eye(4096, device=device, dtype = torch.float64)).item()
smel = loss_smel(torch.eye(4096, device=device, dtype = torch.float64)).item()
print("System finite temperature energy: ", sel)
print("System minimum energy: ", smel)

System finite temperature energy:  10.788146799451779
System minimum energy:  8.195037963688478


## SML optimization 

We optimized minimum eigen value

In [25]:
loss_smel.initializer(compiled_model())
sel = loss_sel(compiled_model()).item()
smel = loss_smel(compiled_model()).item()
print("System finite temperature energy: ", sel)
print("System minimum energy: ", smel)

System finite temperature energy:  10.690365032623017
System minimum energy:  7.985082781440802


## SEL optimization


In [37]:
u1 = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
model2 = rms_torch.UnitaryRieman(H.shape[0], 8, device=device, u0=u1).to(device)
compiled_model2 = torch.compile(model2, dynamic = False, fullgraph=True)

loss_smel.initializer(compiled_model2())
sel = loss_sel(compiled_model2()).item()
smel = loss_smel(compiled_model2()).item()
print("System finite temperature energy: ", sel)
print("System minimum energy: ", smel)

System finite temperature energy:  10.699970951642108
System minimum energy:  8.030657783532432
